In [1]:
from psaw import PushshiftAPI    #library Pushshift
import datetime as dt            #library for date management
#import p                         #library for data manipulation
#import matplotlib.pyplot as plt  #library for plotting
import re
import pandas as pd
#import numpy as np

In [43]:
# load stock list and do with regex check if it is in the post
# mit set und intersection kram
stock_ticker_list = pd.read_excel("../data/external/stock_ticker_list.xlsx")["symbol"].to_list()

In [66]:
words_re = re.compile("|".join(stock_ticker_list))

match = re.search(fr'{"|".join(stock_ticker_list)}', 'Original TSLA DD - Part 2 - Update and Outlook $EBS $BAVA')

if match:
    print(match.start())

0


In [65]:
dir(match)

['__class__',
 '__class_getitem__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'end',
 'endpos',
 'expand',
 'group',
 'groupdict',
 'groups',
 'lastgroup',
 'lastindex',
 'pos',
 're',
 'regs',
 'span',
 'start',
 'string']

In [75]:
"GOEV" in "|".join(stock_ticker_list)

False

In [63]:
post = "Original TSLA DD - Part 2 - Update and Outlook $EBS $BAVA"

match = re.search(fr'{"|".join(stock_ticker_list)}', post)
if match:
    print("found!")
    print(match.group())

found!
O


In [2]:
post = "Original Monkeypox DD - Part 2 - Update and Outlook $EBS $BAVA"

match = re.search(r'\$[A-Z]{1,10}', post)
if match:
    print("found!")
    print(match.group())

found!
$EBS


In [3]:
len(" $GME ")

6

In [18]:
api = PushshiftAPI()              # Object of the API

In [19]:
def data_prep_posts(subreddit, start_time, end_time, filters):

    posts = list(api.search_submissions(
        subreddit=subreddit,   # Subreddit we want to audit
        after=start_time,      # Start date
        before=end_time,       # End date
        filter=filters))       # Column names we want to retrieve

    df = pd.DataFrame(posts)
    df["type"] = "submission"
    return df #Return dataframe for analysis

In [26]:
subreddit = "wallstreetbets"     #Subreddit we are auditing
start_time = int(dt.datetime(2021, 1, 2).timestamp())
#Starting date for our search
end_time = int(dt.datetime(2021, 1, 3).timestamp())
#Ending date for our search
filters = ['id', 'author', 'created_utc',
           'title', "selftext"]
#limit = 1000                     #Elelemts we want to recieve

"""Here we are going to get subreddits for a brief analysis"""
#Call function for dataframe creation of comments
df_p = data_prep_posts(subreddit,start_time,
                       end_time,filters)

In [33]:
def join_submission_title_and_body(title: str, body: str):

    if body != "[removed]":
        return f"{title} {body}"
    else:
        return title

In [34]:
df_p

,author,created_utc,id,selftext,title,created,d_,type
0,Keep_It_Toasty,1609628291,kp7v52,,Good scripture reading for the day #StonksOnly...,1609621091.0,"{'author': 'Keep_It_Toasty', 'created_utc': 16...",submission
1,DesperateSalad5981,1609628267,kp7uve,,Thank you based House for providing us GME buy...,1609621067.0,"{'author': 'DesperateSalad5981', 'created_utc'...",submission
2,Loess_inspired,1609628134,kp7tfw,,Reddit knows I started trading options thanks WSB,1609620934.0,"{'author': 'Loess_inspired', 'created_utc': 16...",submission
3,KYJELLYTIME69,1609628119,kp7t9u,I've been noticing some GME bears coming out o...,"GME DD from a retard bull - The credit market,...",1609620919.0,"{'author': 'KYJELLYTIME69', 'created_utc': 160...",submission
4,ouroboros2decimal718,1609628054,kp7sga,,🌈🐻’s should thank RH for the DD,1609620854.0,"{'author': 'ouroboros2decimal718', 'created_ut...",submission
...,...,...,...,...,...,...,...,...
626,thetrollfromabove,1609542233,kol9iu,[removed],just bought 10 000 shares of MGM by accident a...,1609535033.0,"{'author': 'thetrollfromabove', 'created_utc':...",submission
627,NejKaj,1609542226,kol9fs,[removed],I want to be a degenerate,1609535026.0,"{'author': 'NejKaj', 'created_utc': 1609542226...",submission
628,The-Saint-Lee,1609542217,kol9bk,,How my 2020 went (started April 27),1609535017.0,"{'author': 'The-Saint-Lee', 'created_utc': 160...",submission
629,jeffreymeltz,1609542052,kol7ca,,The Golden Egg,1609534852.0,"{'author': 'jeffreymeltz', 'created_utc': 1609...",submission


In [36]:
df_p["post"] = df_p[["title", "selftext"]].apply(lambda x: join_submission_title_and_body(*x), axis=1)

In [38]:
df_p
# filter correct columns

,author,created_utc,id,selftext,title,created,d_,type,post
0,Keep_It_Toasty,1609628291,kp7v52,,Good scripture reading for the day #StonksOnly...,1609621091.0,"{'author': 'Keep_It_Toasty', 'created_utc': 16...",submission,Good scripture reading for the day #StonksOnly...
1,DesperateSalad5981,1609628267,kp7uve,,Thank you based House for providing us GME buy...,1609621067.0,"{'author': 'DesperateSalad5981', 'created_utc'...",submission,Thank you based House for providing us GME buy...
2,Loess_inspired,1609628134,kp7tfw,,Reddit knows I started trading options thanks WSB,1609620934.0,"{'author': 'Loess_inspired', 'created_utc': 16...",submission,Reddit knows I started trading options thanks ...
3,KYJELLYTIME69,1609628119,kp7t9u,I've been noticing some GME bears coming out o...,"GME DD from a retard bull - The credit market,...",1609620919.0,"{'author': 'KYJELLYTIME69', 'created_utc': 160...",submission,"GME DD from a retard bull - The credit market,..."
4,ouroboros2decimal718,1609628054,kp7sga,,🌈🐻’s should thank RH for the DD,1609620854.0,"{'author': 'ouroboros2decimal718', 'created_ut...",submission,🌈🐻’s should thank RH for the DD
...,...,...,...,...,...,...,...,...,...
626,thetrollfromabove,1609542233,kol9iu,[removed],just bought 10 000 shares of MGM by accident a...,1609535033.0,"{'author': 'thetrollfromabove', 'created_utc':...",submission,just bought 10 000 shares of MGM by accident a...
627,NejKaj,1609542226,kol9fs,[removed],I want to be a degenerate,1609535026.0,"{'author': 'NejKaj', 'created_utc': 1609542226...",submission,I want to be a degenerate
628,The-Saint-Lee,1609542217,kol9bk,,How my 2020 went (started April 27),1609535017.0,"{'author': 'The-Saint-Lee', 'created_utc': 160...",submission,How my 2020 went (started April 27)
629,jeffreymeltz,1609542052,kol7ca,,The Golden Egg,1609534852.0,"{'author': 'jeffreymeltz', 'created_utc': 1609...",submission,The Golden Egg


In [32]:
df_p["selftext"].value_counts()

In [ ]:
dt.datetime(2021, 5, 2).day

In [ ]:
df_p

In [36]:
print(dt.datetime.fromtimestamp(start_time))
print(dt.datetime.fromtimestamp(end_time))

2021-01-01 00:00:00
2021-01-02 00:00:00


In [37]:
df_p["time"] = df_p["d_"].apply(lambda x: dt.datetime.fromtimestamp(x["created_utc"]))

In [39]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   author        604 non-null    object        
 1   created_utc   604 non-null    int64         
 2   domain        604 non-null    object        
 3   id            604 non-null    object        
 4   num_comments  604 non-null    int64         
 5   title         604 non-null    object        
 6   url           604 non-null    object        
 7   created       604 non-null    float64       
 8   d_            604 non-null    object        
 9   time          604 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 47.3+ KB


In [13]:
"""FOR COMMENTS"""
def data_prep_comments(term, start_time, end_time, filters, limit):
    if (len(filters) == 0):
        filters = ['id', 'author', 'created_utc',
                   'body', 'permalink', "score"]
        #We set by default some usefull columns

    comments = list(api.search_comments(
        q=term,                 #Subreddit we want to audit
        after=start_time,       #Start date
        before=end_time,        #End date
        filter=filters,         #Column names we want to retrieve
        limit=limit))           #Max number of comments
    return pd.DataFrame(comments) #Return dataframe for analysis

In [16]:
"""Here we are going to get comments for a brief analysis"""
term = 'wallstreetbets'            #Term we want to search for
limit = 1000                #Number of elelemts
filters = list()
df_c = data_prep_comments(term, start_time,
                          end_time, filters, limit)
#Call function for dataframe creation of comments

In [17]:
df_c

,author,body,created_utc,id,permalink,score,created,d_
0,AleksAFG,Congrats and fuck you. Hopefully I'll be there...,1609541996,ghrq1lb,/r/Bitcoin/comments/kobrs1/my_friends_mocked_m...,1,1.609535e+09,"{'author': 'AleksAFG', 'body': 'Congrats and f..."
1,seb_dm,/r wallstreetbets,1609541987,ghrq111,/r/nevertellmetheodds/comments/ko6ets/the_last...,1,1.609535e+09,"{'author': 'seb_dm', 'body': '/r wallstreetbet..."
2,AutoModerator,This post was flaired as a YOLO so it's on the...,1609541639,ghrpea5,/r/wallstreetbets/comments/kol2v7/to_dee_moon/...,1,1.609534e+09,"{'author': 'AutoModerator', 'body': 'This post..."
3,brodega,"Yeah, all these poor people should just become...",1609541379,ghroxdm,/r/funny/comments/kogdjd/walmart_has_a_plan_fo...,15,1.609534e+09,"{'author': 'brodega', 'body': 'Yeah, all these..."
4,AutoModerator,This post was flaired as a YOLO so it's on the...,1609541147,ghroi46,/r/wallstreetbets/comments/kokxko/golden_egg/g...,1,1.609534e+09,"{'author': 'AutoModerator', 'body': 'This post..."
...,...,...,...,...,...,...,...,...
478,AutoModerator,Your post has been removed because a single li...,1609455994,ghnh6e9,/r/wallstreetbets/comments/ko0556/the_truth_ab...,1,1.609449e+09,"{'author': 'AutoModerator', 'body': 'Your post..."
479,AutoModerator,This post was flaired as a YOLO so it's on the...,1609455976,ghnh5ad,/r/wallstreetbets/comments/ko04yx/gme_yolo_upd...,1,1.609449e+09,"{'author': 'AutoModerator', 'body': 'This post..."
480,talesofstocks,"lol Oh no, the downvotes from reddit users in ...",1609455819,ghngw1u,/r/wallstreetbets/comments/knus1d/it_iz_what_i...,0,1.609449e+09,"{'author': 'talesofstocks', 'body': 'lol Oh no..."
481,AutoModerator,"Sir, this is the unemployment line.\n\n*I am a...",1609455753,ghngs64,/r/wallstreetbets/comments/ko02mo/crm_salesfor...,1,1.609449e+09,"{'author': 'AutoModerator', 'body': 'Sir, this..."


In [46]:
df_c

,author,body,created_utc,id,permalink,subreddit,created,d_
0,AleksAFG,Congrats and fuck you. Hopefully I'll be there...,1609541996,ghrq1lb,/r/Bitcoin/comments/kobrs1/my_friends_mocked_m...,Bitcoin,1.609538e+09,"{'author': 'AleksAFG', 'body': 'Congrats and f..."
1,seb_dm,/r wallstreetbets,1609541987,ghrq111,/r/nevertellmetheodds/comments/ko6ets/the_last...,nevertellmetheodds,1.609538e+09,"{'author': 'seb_dm', 'body': '/r wallstreetbet..."
2,AutoModerator,This post was flaired as a YOLO so it's on the...,1609541639,ghrpea5,/r/wallstreetbets/comments/kol2v7/to_dee_moon/...,wallstreetbets,1.609538e+09,"{'author': 'AutoModerator', 'body': 'This post..."
3,brodega,"Yeah, all these poor people should just become...",1609541379,ghroxdm,/r/funny/comments/kogdjd/walmart_has_a_plan_fo...,funny,1.609538e+09,"{'author': 'brodega', 'body': 'Yeah, all these..."
4,AutoModerator,This post was flaired as a YOLO so it's on the...,1609541147,ghroi46,/r/wallstreetbets/comments/kokxko/golden_egg/g...,wallstreetbets,1.609538e+09,"{'author': 'AutoModerator', 'body': 'This post..."
...,...,...,...,...,...,...,...,...
478,AutoModerator,Your post has been removed because a single li...,1609455994,ghnh6e9,/r/wallstreetbets/comments/ko0556/the_truth_ab...,wallstreetbets,1.609452e+09,"{'author': 'AutoModerator', 'body': 'Your post..."
479,AutoModerator,This post was flaired as a YOLO so it's on the...,1609455976,ghnh5ad,/r/wallstreetbets/comments/ko04yx/gme_yolo_upd...,wallstreetbets,1.609452e+09,"{'author': 'AutoModerator', 'body': 'This post..."
480,talesofstocks,"lol Oh no, the downvotes from reddit users in ...",1609455819,ghngw1u,/r/wallstreetbets/comments/knus1d/it_iz_what_i...,wallstreetbets,1.609452e+09,"{'author': 'talesofstocks', 'body': 'lol Oh no..."
481,AutoModerator,"Sir, this is the unemployment line.\n\n*I am a...",1609455753,ghngs64,/r/wallstreetbets/comments/ko02mo/crm_salesfor...,wallstreetbets,1.609452e+09,"{'author': 'AutoModerator', 'body': 'Sir, this..."


In [47]:
# get the comments of a certain submissions by some kind of id and join them together in database